In [1]:
%matplotlib notebook
import os

import jplus
#jplus_dir         =  '/home/CEFCA/aaorsi/work/j-plus/'
elg_analysis_dir  = '/home/CEFCA/aaorsi/work/elg_jplus/'
elgdata           = '%s/out/elgs.dat' % elg_analysis_dir
redmapper_dir     = '/home/CEFCA/aaorsi/work/redmapper/'
redmapperdata     = redmapper_dir + 'redmapper_dr8_public_v6.3_catalog.fits'
#tilesdata         = '%s/tiles/tiles_data_new.tsv' % elg_analysis_dir


cwd = os.getcwd()
import sys

#sys.path.append(jplus_dir)
sys.path.append(elg_analysis_dir)

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from astropy.io import fits

#import elg_analysis as elg
import elgtools as tools_elg
import learn_elgs as learn_elg
import pickle

from astropy import units as u
from astropy.coordinates import SkyCoord

tile_scale = 1.40

rmin = -2
rmax =np.log10(tile_scale/2.)
nbins = 20

rarr = np.linspace(rmin, rmax, nbins)
dr = rarr[1] - rarr[0]

#t_info = np.loadtxt(tilesdata) ## Old tile list info, don't use
#print 'tiles info read'
tiles = jplus.datasets.fetch_jplus_tile_list(db='test2',overwrite=False)


j-plus [INFO]: Fetching J-PLUS Tile list
j-plus [INFO]:    Loading /home/CEFCA/aaorsi/photoz/jplus_data/objects_tile_list_test2_dual.h5


In [2]:
tiles.keys()


['ncombined',
 'noise',
 'min_ra',
 'ADUlevel',
 'tileIDs',
 'texposed',
 'min_dec',
 'ref_tileID',
 'filter',
 'depth',
 'effectime',
 'max_dec',
 'ra',
 'SQL',
 'date',
 'max_ra',
 'dec',
 'fwhmg']

In [3]:
def get_distance(ra1, dec1, ra2, dec2):
  ra1 *= np.pi/180.
  ra2 *= np.pi/180.
  dec1 *= np.pi/180.
  dec2 *= np.pi/180.
  
# Angular distance for two sets of coordinates
  cosg = (np.cos(np.pi/2. - dec1)*np.cos(np.pi/2. - dec2) +
          np.sin(np.pi/2. - dec1)*np.sin(np.pi/2.-dec2)*np.cos(ra1 - ra2))
  
  return np.arccos(cosg)


def quick_dist(ra1, dec1, ra2, dec2, units='deg'):
  return np.sqrt( ((ra1 - ra2)*np.cos(dec1*np.pi/180))**2 + (dec1 - dec2)**2)

def haversine_dist(ra1, dec1, ra2, dec2):
  th1 = np.pi/2. - dec1 * np.pi/180.0
  th2 = np.pi/2. - dec2 * np.pi/180.0

  ph1 = ra1 * np.pi/180.0
  ph2 = ra2 * np.pi/180.0

  dph = np.abs(ph1 - ph2)
  dth = np.abs(th1 - th2)

  harg = np.sin(dph/2)**2 + np.cos(ph1)*np.cos(ph2) * np.sin(dth/2.)**2

  return 2 *np.arcsin(np.sqrt(harg)) * 180./np.pi  # Return distance in degrees


def skydist(ra1,dec1,ra2,dec2, units='deg'):
  
  n2 = len(ra2)

  if n2 != len(dec2):
    raise ValueError('ra, dec arrays have different number of elements')
  
  dist = np.zeros(n2)

  c1 = SkyCoord(ra1,dec1,unit=units)
  for i in range(n2):
    c2 = SkyCoord(ra2[i], dec2[i], unit=units)
    dist[i] = c1.separation(c2).radian

  return dist

def load_catalogues(elgfile = elgdata, centralobj = redmapperdata, centralobjtype = 'fits', 
                    zrange=[.3,.35]):


# Function to load elgs and redmapper catalogues

  elgs = pickle.load(open(elgfile))
  print 'ELG catalogue loaded'

  if centralobjtype == 'fits':
    raw_central = fits.open(centralobj)
    central0     = raw_central[1].data
    zsel = np.where((central0['z_lambda'] > zrange[0]) & (central0['z_lambda'] < zrange[1]))[0]
    ncentral = len(zsel)
    central = central0[zsel]
  else:
    central = centralobj
    ncentral = len(central['ra'])
    zsel     = np.arange(ncentral)

  
  return {'elgs':elgs, 'central':central}

  
def get_density(datadic, tiles, dfunc = haversine_dist):  
  
  gal_arr = np.zeros(nbins)
  central = datadic['central']
  elgs    = datadic['elgs']
  ncentral  = len(central['ra'])
  nelgs    = len(elgs)
    
  # dfunc = quick_dist  # Define which distance calculator to use
  # dfunc = haversine_dist # get_distance  # Define which distance calculator to use
  
  print 'central objects loaded, %d clusters' % ncentral

  # Select only those central objects 
  # near a J-PLUS tile
  
  
  elgtiles = np.unique(elgs['tile_id'])
        
  ntiles = len(elgtiles)
  print 'ELG sample is contained in %d tiles' % ntiles
  print 'scanning tiles..'
 
  ktot = 0
  for i in range(ncentral):
    idc = i

    dist = dfunc(central['ra'][idc], central['dec'][idc], tiles['ra'], 
                tiles['dec'])
    idist = np.argmin(dist)
    mindist = dist[idist]
    
    if mindist < tile_scale:
      idtt = np.where(elgtiles == tiles['tileIDs'][idist])[0] # Tile with central obj contains ELGs
      
      if len(idtt) != 1:
        continue # len(idtt) = 0 would mean that this tile is empty -- not observed or bad photometry

      sel_tile = np.where(elgs['tile_id'] == tiles['tileIDs'][idist])[0]
      ngals = len(sel_tile)
  
      ktot += 1.0 # Tile contains central galaxy + ELGs around it.

      for k in range(ngals):
        idg = sel_tile[k]
        dist = np.log10(dfunc(central['ra'][idc], central['dec'][idc], 
        elgs['coords'][idg,0], elgs['coords'][idg,1]))

        if dist < rmax:
          dbin = int((dist  - rmin)/dr)
          gal_arr[dbin:] += 1

#    if ktot == 79:
#      break

  print gal_arr/ktot

  print 'N centers: ',ktot
  return gal_arr/ktot


def plot_densities(d1,label):
  import matplotlib.pyplot as plt
  vol_r = 1 #np.pi*(10**rarr)**2

  for d, l in zip(d1, label):
      plt.semilogy(rarr, d/vol_r,label=l,linewidth=3)

  plt.legend(fontsize=20)

  plt.xlabel(r'$\log(\theta [{\rm deg}])$',fontsize=20)
  plt.ylabel(r'$\langle n \rangle (\leq r) [{\rm deg}^{-2}]$',fontsize=20)

  plt.xlim([rarr.min(),rarr.max()])
  #plt.ylim([np.min([d1v,d2v]), d2v.max()])

  plt.show()

  return 1


In [ ]:

#def elg_clusters():

CreateRandoms = True
RanFile = 'random_mask.dat'

mastercat = load_catalogues()
tcenter = {'ra':tiles['ra'], 'dec':tiles['dec']}
tilecat = load_catalogues(centralobj=tcenter, centralobjtype='')


elgs = mastercat['elgs']
import pymangle

tiles = jplus.datasets.fetch_jplus_tile_list(db='test2',overwrite=False)
stars = jplus.datasets.fetch_jplus_objects_to_mask(db='test2',overwrite=False)
mask  = jplus.mangle.Mask(tiles,stars, overwrite=False)


if CreateRandoms:
    print 'Creating random mask file...'
    ran = mask.create_random(1e6)
    with open(RanFile,'wb') as outfile:
        pickle.dump(ran,outfile,protocol=pickle.HIGHEST_PROTOCOL)
else:
    print 'Reading random mask file ...'
    ran = pickle.load(RanFile)
    
                
good = mask.contains(elgs['coords'])

nran = len(ran['coords'])
ran_tile_id = np.zeros(nran)
for i in range(nran):
    rc = ran['coords']
    dist = haversine_dist(rc[i,0], rc[i,1], tiles['ra'], 
                tiles['dec'])
    idist = np.argmin(dist)
    mindist = dist[idist]
    ran_tile_id[i] = tiles['tileIDs'][idist]


randict = {'coords':ran['coords'], 'tile_id':ran_tile_id}    

mask_clusters = {}  # Mask density around ELGs
mask_clusters['elgs'] = randict
mask_clusters['central'] = mastercat['central']


mask_tile = {}
mask_tile['elgs'] = randict
mask_tile['central'] = tilecat['central']


  

j-plus [INFO]: Fetching J-PLUS Tile list
j-plus [INFO]:    Loading /home/CEFCA/aaorsi/photoz/jplus_data/objects_tile_list_test2_dual.h5
j-plus [INFO]:    Loading /home/CEFCA/aaorsi/photoz/jplus_data/jplus_objects_to_mask_test2_dual.h5
j-plus [INFO]: Reading Mangle Mask /home/CEFCA/aaorsi/photoz/jplus_data/mask_data/jplus_mask.pol
j-plus [INFO]: Creating random points


In [ ]:
os.path.expandvars('$MANGLEBINDIR')


In [ ]:
hp = jplus.healpix.HealpixMap(elgs['coords'][:,:], average=False, nside=256)#, value=obj_jplus['rJAVA'][good,0])
hmap = hp.plot(interactive=False, title="Average J0660 magnitude")

In [ ]:
hp = jplus.healpix.HealpixMap(randict['coords'][:,:], average=False, nside=256)#, value=obj_jplus['rJAVA'][good,0])
hmap = hp.plot(interactive=False, title="Average J0660 magnitude")

In [ ]:
print g4

In [ ]:
plt.figure(12)
plt.plot(randict['coords'][:,0],randict['coords'][:,1], '.',color='gray')
plt.plot(elgs['coords'][:,0],elgs['coords'][:,1], ',',color='yellow')
plt.show()



In [ ]:
g1 = get_density(mastercat, tiles)
g2 = get_density(tilecat, tiles)

g3 = get_density(mask_clusters, tiles)
g4 = get_density(mask_tile, tiles)



In [ ]:
plt.figure(11)
plot_densities([g1,g2,g3,g4],['clusters','tile centers', 'mask', 'mask around tile center'])
plt.figure(22)
plot_densities([g1/g3,g2/g4],['clusters','tile centers'])

In [ ]:
plt.figure()

tnumber = 0
tid = tiles['tileIDs'][tnumber]
print tid
mask = randict['tile_id'] == randict['tile_id'][0]
print mask[mask == True]
plt.plot(ran['coords'][mask,0]-tiles['ra'][tnumber],ran['coords'][mask,1]-tiles['dec'][tnumber],'o')

In [ ]:
tile, mag, total = jplus.plotting.groupbytile(elgs['tile_id'], elgs['J0660'][:,0])
plt.figure()
plt.plot(tile, mag,'o')
print tile, total

In [ ]:
elgs[]